# StarGAN2 operations

This Colab is from repository https://github.com/eps696/stargan2.  
Please read instructions and hints there, regarding data preparation and training process.



**Run this cell after each session restart**

In [ ]:
#@title General setup { display-mode: "form", run: "auto" }

!pip install gputil ffpb

import os
from base64 import b64encode

import ipywidgets as ipy
from IPython.display import HTML, Image, display, clear_output
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# from google.colab import output, files

import warnings
warnings.filterwarnings("ignore")

!apt-get -qq install ffmpeg
from google.colab import drive
drive.mount('/G', force_remount=True)
# gdir = !ls /G/
# gdir = '/G/%s/' % str(gdir[0])
gdir = '/G/MyDrive/'
%cd $gdir

#@markdown Copying StarGAN2 to the directory below on your Google drive (creating it, if it doesn't exist):
work_dir = 'stargan2' #@param {type:"string"}
#@markdown NB: All paths below are relative to this directory (except the archive with source images on the next step). 

#@markdown NB: Avoid connecting Google drive manually via the icon in Files section on the left. Doing so may break further operations.

work_dir = os.path.join(gdir, work_dir)
if not os.path.isdir(work_dir):
  !git clone git://github.com/eps696/stargan2 $work_dir
%cd $work_dir
!pip install -r requirements.txt

clear_output()

from src.utilgan import file_list, img_list, basename
model = ''
def model_select(work_dir):
  models = file_list(work_dir, 'ckpt', subdir=True)
  models = [m.replace(work_dir, '') for m in models if not 'optims' in basename(m)]
  global model
  model = models[0]
  def on_change(change):
    global model
    if change['type'] == 'change' and change['name'] == 'value':
      model = change['new']
      if model[0]=='/': model = model[1:]
      model = os.path.join(work_dir, model)
      print('.. selected model', model)
  model_select = ipy.Dropdown(options=models, description='Found models:', style={'description_width': 'initial'}, layout={'width': 'max-content'})
  display(model_select)
  model_select.observe(on_change)
# model_select(work_dir)

def makevid(seq_dir, size=None):
  char_len = len(basename(img_list(seq_dir)[0]))
  out_sequence = seq_dir + '/%0{}d.jpg'.format(char_len)
  out_video = seq_dir + '.mp4'
  !ffpb -y -i $out_sequence -crf 18 $out_video
  data_url = "data:video/mp4;base64," + b64encode(open(out_video,'rb').read()).decode()
  wh = '' if size is None else 'width=%d height=%d' % (size, size)
  return """<video %s controls><source src="%s" type="video/mp4"></video>""" % (wh, data_url)

# Hardware check
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
import GPUtil as GPU
gpu = GPU.getGPUs()[0]
!nvidia-smi -L
print("GPU RAM {0:.0f}MB | Free {1:.0f}MB)".format(gpu.memoryTotal, gpu.memoryFree))
print('\nDone!')

## Training

First, let's prepare the multi-domain dataset. Ensure all your images are RGB (3 channels). Ensure the minimum size (by any side) is bigger than model resolution below.
Collect your dataset as described [in the repo](https://github.com/eps696/stargan2).

Upload zip-archive with images onto Google drive and type its path below (relative to G-drive root). Run cell below every time, once you get new Colab runtime (cause we place the dataset on local disc for maximum speed).

In [ ]:
#@title Data setup 
dataset = 'test' #@param {type:"string"}
source = 'stargan2/data/test.zip' #@param {type:"string"}
data_dir = os.path.join('/content', dataset)

# cleanup previous attempts
![ -d "/content/tmp" ]; rm -rf /content/tmp 
![ -d $data_dir ]; rm -rf $data_dir

!mkdir /content/tmp
%cd /content/tmp
fpath = os.path.join(gdir, source)
!unzip -o -q $fpath
unpack_dir = os.path.join('/content/tmp', basename(source))
!mv $unpack_dir $data_dir
%cd $work_dir
!ls $data_dir

Now, we can train StarGAN2 on the prepared dataset.

In [ ]:
#@title Train
%cd $work_dir
dataset = 'test' #@param {type:"string"}
size = 256 #@param [256,512]
batch =  6#@param {type:"integer"}
lambda_ds = 2. #@param {type:"number"}
lambda_cyc = 1. #@param {type:"number"}
lambda_sty = 1. #@param {type:"number"}
steps_k = 100 #@param {type:"integer"}
sample_every_k =  0.01#@param {type:"number"}
save_every_k = 0.05 #@param {type:"number"}
resume = 0 #@param {type:"integer"}

# data_dir = os.path.join(data_dir, 'data', dataset)
model_dir = os.path.join(work_dir, 'train', dataset)
img_size = int(size)
steps = steps_k * 1000
sample_every = sample_every_k * 1000
save_every = save_every_k * 1000

%run src/train.py --data_dir $data_dir --model_dir $model_dir --img_size $img_size --batch $batch --total_iters $steps --sample_every $sample_every --save_every $save_every --lambda_ds $lambda_ds --lambda_cyc $lambda_cyc --lambda_sty $lambda_sty --resume $resume

> This will run training process, according to the options in `src/train.py`. 
There are two types of models, saved under `train/<dataset>` directory: 
* generators = named as `<dataset>-<size>-<domaincount>-<kiloiters>.pkl` (e.g. `test-256-7-360.pkl`), 
* full set (suffixed as `nets`/`optims`) to resume training.  
Full models are saved every `save_every_k` thousand steps; test samples and generators - every `sample_every_k` thousand steps. Test samples are saved under `train/<dataset>/test`. Check them to follow the progress! 

> Training duration is defined by `steps_k` (thousands of steps). Reasonable length for batch=6 (which is maximum for size 256 on standard Colab GPU) is 100-120k, taking 5-7 days. Increase batch count if you get GPU with > 16gb RAM. 

> Set `resume` to the kilo-iterations of the last full model in the training directory to resume from it. Since the optimizer state is also saved, stopping/resuming does not harm the training process.

> **NB: Saved models can quickly occupy disk space, watch out for them!**

Don't forget to read [some comments/findings](https://github.com/eps696/stargan2) about training details.

Other training options:

In [ ]:
%run src/train.py --help

In [ ]:
#@title ### Tweak models

#@markdown One can mix few models by stochastic averaging all weights:

models_dir = 'models' #@param {type:"string"}

%run src/swa.py --in_dir $models_dir

## Generation

Let's produce some imagery from the original `afhq` model (get it [here](https://www.dropbox.com/s/etwm810v25h42sn/100000_nets_ema.ckpt?dl=0) and put onto Google drive somewhere under our working directory).  

In [ ]:
#@title ### Generator setup
!cd $work_dir

#@markdown Run this cell and select model from the dropdown form below:
print(work_dir)
model_select(work_dir)
if model[0]=='/': model = model[1:]
model = os.path.join(work_dir, model)
print('.. selected model', model)

In [ ]:
#@title ### Single images processing

#@markdown Put image files in the `_in` directory (under our working one) and list `refs` (numbers of domains), separated by dash. This would produce 3 images (one per trained domain in the model) for every input image. Results are saved in the `_out` directory.  

refs = '0-1-2' #@param {type:"string"}
%cd $work_dir
%run src/test.py --source _in --model $model --refs $refs

# ipython_display(ImageSequenceClip(img_list(out_dir), fps=25), center=False)

In [ ]:
#@title ### Image animation

#@markdown Process single image [*put your own file*], interpolating between referenced domains, with specific total duration.

image_in = '_in/test.jpg' #@param {type:"string"}
refs = '0-1-2' #@param {type:"string"}
frames = 100 #@param {type:"integer"}
out_dir = '_out/animation' #@param {type:"string"}

%cd $work_dir
%run src/process.py --source $image_in --model $model --out_dir $out_dir --frames $frames --refs $refs
HTML(makevid(os.path.join(out_dir, basename(image_in))))

In [ ]:
#@title ### Video processing

#@markdown Process video file [*put your own file*], interpolating between referenced domains.

video_in = '_in/test.mp4' #@param {type:"string"}
refs = '0-1-2' #@param {type:"string"}
out_dir = '_out/video' #@param {type:"string"}

in_tmp = os.path.join('_in', basename(video_in))
out_tmp = os.path.join(out_dir, basename(video_in))
in_ff = os.path.join(in_tmp, '%06d.jpg')

%cd $work_dir
os.makedirs(in_tmp, exist_ok=True)
os.makedirs(out_tmp, exist_ok=True)

!ffmpeg -y -v warning -i $video_in $in_ff
%run src/process.py --source $in_tmp --model $model --out_dir $out_dir --refs $refs
HTML(makevid(out_tmp))

In [ ]:
#@title ### Recurrent generation

#@markdown Generate video sequence, interpolating between referenced domains in a feedback loop, switching domain every `fstep` frames.

sizeX = 1280 #@param {type:"integer"}
sizeY = 720 #@param {type:"integer"}
refs = '0-1-2' #@param {type:"string"}
frames = 100 #@param {type:"integer"}
fstep = 25 #@param {type:"integer"}
out_dir = '_out/recurs' #@param {type:"string"}

%cd $work_dir
%run src/process.py --model $model --refs $refs --size $sizeX-$sizeY --frames $frames --fstep 25 --out_dir $out_dir --recurs 1
HTML(makevid(out_dir))

In [ ]:
#@title ### Recurrent drawing over mask

#@markdown Generate similar video sequence, drawing over contour mask (useful for videomapping projections):

mask_image = '_in/mapping.jpg' #@param {type:"string"}
refs = '0-1-2' #@param {type:"string"}
frames = 100 #@param {type:"integer"}
fstep = 25 #@param {type:"integer"}
recurrence = 0.4 #@param {type:"number"}
sizeX = 1280 #@param {type:"integer"}
sizeY = 720 #@param {type:"integer"}
out_dir = '_out/mapping' #@param {type:"string"}

%cd $work_dir
%run src/process.py --source $mask_image --model $model --refs $refs --size $sizeX-$sizeY --frames $frames --fstep 25 --out_dir $out_dir --recurs $recurrence
HTML(makevid(os.path.join(out_dir, basename(mask_image))))